In [2]:
import random
import pandas
import numpy
from sklearn.preprocessing import OneHotEncoder
import random
import itertools

## Input Dataset

In [3]:
dataset = pandas.read_csv("dataset.csv")
dataset.head()

,Suhu,Waktu,Kondisi Langit,Kelembapan,Terbang/Tidak
0,Normal,Siang,Berawan,Normal,Tidak
1,Normal,Siang,Cerah,Normal,Tidak
2,Normal,Pagi,Hujan,Normal,Tidak
3,Normal,Sore,Rintik,Rendah,Ya
4,Normal,Malam,Berawan,Tinggi,Tidak


## Input Datatrain

In [4]:
datatrain = OneHotEncoder(sparse=False).fit_transform(dataset).astype(int)
datatrain = numpy.delete(datatrain,14,1)
pandas.DataFrame(datatrain).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0
1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0
2,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0
3,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1
4,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0


## Init First Generation

In [5]:
def initPopulis(populationNumber, chromosomeSize):
    populisInit = []
    for i in range(0, populationNumber):
        chromosome = createChromosome(chromosomeSize)
        populisInit.append(numpy.array(chromosome))
    return populisInit

def createChromosome(chromosomeSize):
    chromosome = []
    for i in range(0, chromosomeSize):
        chromosome.append(random.randint(0, 1))
    return chromosome
populis = initPopulis(10,30)

## Split Array by Rule

In [6]:
def split(arr):
    splitted = []
    for rule in arr:
        splitted.append(numpy.split(rule,[3,7,11,14]))
    return splitted   

## Calculate Fitness

In [7]:
def calc_fitness(data_train,population):
    fitness = []
    h = 0
    split_data_train = split(data_train)
    for chrom in population:
        # split # each chromosome can contain more than 1 rule
        i = 0
        k = 0
        subSplitPopulation = numpy.split(population[h],round(len(population[h])/15))
        for data in split_data_train:
            samaAll = False
            adaBeda = False
            j=0
            for krom in split_data_train[k]:
                for m in range(len(subSplitPopulation)):
                    rulePop = split(subSplitPopulation)
#                     print(numpy.all( ~(numpy.logical_xor(rulePop[0][j],split_data_train[k][j]))))
                    if numpy.all( ~(numpy.logical_xor(rulePop[0][j],split_data_train[k][j]))):
                        samaAll = True
                        break
                    elif numpy.any(numpy.logical_xor(rulePop[0][j],split_data_train[k][j])):
                        adaBeda = True
                        break
                j +=1
            if samaAll:
                if(numpy.equal(rulePop[0][4],split_data_train[k][4])):
                    i+=1
            elif adaBeda:
                if(numpy.not_equal(rulePop[0][4],split_data_train[k][4])):
                    i+=1
            k+=1
        fit = (i/len(data_train))*100
        fitness.append(fit)
        h+=1
    return fitness
fitness = calc_fitness(datatrain,populis)
fitness

[100.0, 92.5, 73.75, 78.75, 85.0, 77.5, 73.75, 73.75, 100.0, 67.5]

## Parent Selection

In [8]:
def rouletteWheel(pops,fitness):
    fitness_sum = sum(fitness)
    probability_offset = 0

    i = 0
    probability = []
    for pop in pops:
        probs =probability_offset + (fitness[i] / fitness_sum) 
        probability.append(probs)
        probability_offset += probs
        i+=1
    r = random.random()

    i = 0
    parent = 0
    for pop in pops:
        if probability[i] > r:
            break; 
        parent = i
        i+=1
    return parent

In [9]:
def parentSelection(pops,fitness):
    while True:
        parent1idx = rouletteWheel(populis,fitness)
        while True:
            parent2idx = rouletteWheel(populis,fitness)
            if (parent1idx != (parent2idx)):
                break
        break
    return parent1idx,parent2idx

In [10]:
def order1Crossover(parent1,parent2):
    lb1 = random.randint(0,len(parent1)-1)
    ub1 = random.randint(lb1,len(parent1)-1)
    lb2 = random.randint(lb1,ub1)
    ub2 = random.randint(lb2,ub1)
    parentCrossed = []
    for i in range (len(parent1)):
        if i < lb2 :
            parentCrossed.append(parent1[i])
        elif lb2 <= i <= ub2 :
            parentCrossed.append(parent2[i])
        elif ub2 < i:
            parentCrossed.append(parent1[i])
    return parentCrossed

## Crossover

In [11]:
def crossover(parent1,parent2):
#   if the 
    if (len(parent1)>len(parent2)):
        parentTemp = parent1
        parent1 = parent2
        parent2 = parentTemp
    lb1 = random.randint(0,len(parent1)-1)
    point1 = [lb1,random.randint(lb1,len(parent1)-1)]
    
    diff = point1[1]-point1[0]
    gap = diff % 15
    
    p2Candidate = [[point1[0],point1[0]+diff],[point1[0],point1[0]+gap],[point1[1]-diff,point1[1]],[point1[1]-gap,point1[1]]]
    p2Candidate.sort()
    p2Candidate = list(p2Candidate for p2Candidate,_ in itertools.groupby(p2Candidate))
    point2 = random.choice(p2Candidate)
    
    subParent1 = parent1[point1[0]:point1[1]]
    subParent2 = parent2[point2[0]:point2[1]]
    
    parent1 = numpy.hstack([parent1[0:point1[0]],subParent2,parent1[point1[1]:len(parent1)]])
    parent2 = numpy.hstack([parent2[0:point2[0]],subParent1,parent2[point2[1]:len(parent2)]])
    
    parent1.tolist()
    parent2.tolist()
    
    return parent1,parent2

pr1,pr2 = parentSelection(populis,fitness)
crossover(populis[pr1],populis[pr2])

(array([0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 0, 1]),
 array([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1]))

In [12]:
def scrambleMutation(parent):
    lb = random.randint(0,len(parent)-1)
    ub = random.randint(lb,len(parent)-1)
    shuffledParent = []
    probability = 0.1
    alpha = random.uniform(0,1)
    if (alpha <= probability):
#         print('lb',lb)
#         print('ub',ub)
        for i in range (len(parent)):
            if lb <= i <= ub:
                shuffledParent.append(parent[i])
        random.shuffle(shuffledParent)
        for k in range (len(parent)):
            if lb <= k <= ub:
                parent[k] = shuffledParent[k-lb]
    return parent

In [13]:
def survivors(parent1,parent2):
    newGen = initPopulis(8,30)
    newGen.append(parent1)
    newGen.append(parent2)
    return newGen

In [14]:
def perfectFitnessCheck(fitness):
    for i in range(len(fitness)):
        if (fitness[i] >= 100) :
            return i
    i = -1
    return (-1)

In [15]:
def evolve():
    populis = initPopulis(10,30)
    i=-1
    while (i==-1):
        fitness = calc_fitness(datatrain,populis)
        i = perfectFitnessCheck(fitness)
        print(i)
        if (i==-1):
            pr1idx,pr2idx = parentSelection(populis,fitness)
            child1,child2 = crossover(populis[pr1idx],populis[pr2idx])
            newChild1 = scrambleMutation(child1)
            newChild2 = scrambleMutation(child2)
            populis = survivors(newChild1,newChild2)
evolve()

0


In [16]:
pr1,pr2 = parentSelection(populis,fitness)
parent1,parent2 = crossover(populis[pr1],populis[pr2])
parent1 = scrambleMutation(parent1)
parent1

array([0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 1])

In [17]:
arr = numpy.arange(0,45)
print(arr)
rules = numpy.split(arr,round(len(arr)/15))
print(rules)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]), array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])]


In [18]:
a = numpy.array([1,0,0])
b = [1,0,1]
numpy.any(numpy.logical_and(a,b))

True

In [19]:
split(populis)[0][0]
split(datatrain)[0][0]
a = split(populis)[4][0]
b = split(datatrain)[0][0]
numpy.any(numpy.logical_and(a,b))

True

In [20]:
populis[0]

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1])

In [22]:
arr = numpy.concatenate((populis[0],populis[1]))
print(len(arr))
a = len(arr)
rules = numpy.split(arr,2)
print(rules)

60
[array([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1]), array([1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1])]


In [23]:
def calc_fitness(data_train,population):
    fitness = []
    h = 0
    split_data_train = split(data_train)
    split_population = split(population) 
    for chrom in population:
        # split # each chromosome can contain more than 1 rule

        i = 0
        k = 0
        for data in split_data_train:
            sama = False
            j=0
        
            for krom in split_data_train[k]:
                if numpy.any( ~(numpy.logical_xor(split_population[h][j],split_data_train[k][j]))):
                    sama = True
                    break
                j +=1
            if sama:
                if(numpy.equal(split_population[h][4],split_data_train[k][4])):
                    i+=1
            else:
                print("false")
            k+=1
        fitness.append((i/len(data_train))*100)
        h+=1
    return fitness
fitness = calc_fitness(datatrain,populis)
fitness

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()